In [182]:
#Resolving coref for named entities in each news articles and incrementally stroing it in a CSV file

import pandas as pd
import numpy as np
import os
import json
import nltk
import sys
from pycorenlp import StanfordCoreNLP
import corenlp
from nltk import word_tokenize
import spacy
from pprint import pprint
import corenlp
os.environ['CORENLP_HOME'] = '/home/ahaque2/project/virtual_environment_1/stanfordNLP'

In [183]:
data = pd.read_csv('../News_Media_bias/Git Code/Dataset/articles1.csv')
print(data.columns)

Index(['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year',
       'month', 'url', 'content'],
      dtype='object')


In [165]:
news_articles = data['content']

In [166]:
def get_stanford_ann(sent):
    nlp=StanfordCoreNLP("http://localhost:9000/")
    output = nlp.annotate(sent, properties={'annotators': 'tokenize,ssplit,pos,dcoref','timeout': '1000000', 'outputFormat': 'json'})
    return output


In [167]:
# text = ["Bobby thought Bill should buy a trailer and haul it with his car.",
#             "Bill thought a truck would be better for what he needed.",
#             "Bobby pointed out two vehicles were much more expensive.",
#             "Bill was set in his ways with conventional thinking.",
#             "He ended up buying the truck he wanted despite Bobby's advice."]

# text = " ".join(text)
# print(text)

In [168]:
def resolve_coref(corefs, numSent):
    
    coref_resolved = []
    
    for cls in corefs.keys():
        entity = None
        replace_list = []
        if(len(corefs[cls])>1):
            for inst in corefs[cls]:
                if(inst['isRepresentativeMention'] == True):
                    entity = inst['text']
                else:
                    replace_list.append((entity, inst['text'], inst['sentNum'], inst['startIndex'], inst['endIndex']))
            coref_resolved.extend(replace_list)

    
    coref_resolved.sort(key = lambda x: x[2])
    sent_corefs = []
    for i in range(1,numSent + 1):
        l = list(filter(lambda x: x[2] == i, coref_resolved))
        l.sort(key = lambda x: x[3])
        sent_corefs.append(l)
        
    return sent_corefs

def replace_corefs(tok_sent, sent_corefs):
    sent_resolved = []
    for sent, cor in zip(tok_sent, sent_corefs):
        #print("cor ", cor)
        #print(sent)
        i = 0
        final_sent = []
        end_index = 0
        for c in cor:
            if(c[0] != None and len(c[1]) < 30):
                #print("c ", c)
                
                #Replace simultanously only then it will work
                
                start_index, end_index = c[3]-1, c[4]-1
                #print("strat and end indexes ", start_index, end_index, i)
                final_sent = final_sent + sent[i:start_index] + [c[0]]
                #i = len(final_sent)
                i = end_index
                #print("Here ", final_sent)
        final_sent = final_sent + sent[end_index:len(sent)]
        #print("Final sentence ", final_sent)
        #print()
        #sys.exit()
        sent_resolved.extend(final_sent)

    return sent_resolved

def get_resolved_coref_text(text):
    
    stanford_ann = get_stanford_ann(text)
    #tokenized_sent = stnfrd_coref['sentences'][0]['tokens']
    #print(stnfrd_coref)
    #sys.exit()
    corefs = stanford_ann['corefs']
    num_of_sent = len(stanford_ann['sentences'])
    #print(corefs.keys())
    coref_resolved = []
    
    sent_corefs = resolve_coref(corefs, num_of_sent)
    #print("sent_corefs ", sent_corefs)
    tok_sent = []
    ner = {}
    for index in range(num_of_sent):
        tok_sent.append([toks['originalText'] for toks in stanford_ann['sentences'][index]['tokens']])
    
    #ner = get_ner(stanford_ann, num_of_sent)
                
    #print("tok_sent ", tok_sent)
    sent_resolved = replace_corefs(tok_sent, sent_corefs)
    #print(sent_resolved)
    s= " ".join(sent_resolved)
    return s


def get_ner(stanford_ann, num_of_sent):
    
    ner = {}
    for index in range(num_of_sent):
        for toks in stanford_ann['sentences'][index]['tokens']:
                if(toks['ner'] != 'O'):
                    if(toks['ner'] in ner):
                        ner[toks['ner']].append(toks['lemma'])
                    else:
                        ner[toks['ner']] = [toks['lemma']]
                        
    for key in ner:
        ner[key] = set(ner[key])
        
    return ner

def get_spacy_ner(text):

    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    ner = {}
    for ent in doc.ents: 
        if ent.label_ in ner:
            ner[ent.label_].append(ent.text)
        else:
            ner[ent.label_] = [ent.text]
        
    for key in ner:
        ner[key] = set(ner[key])
        
    return ner

def get_data_by_news_media(d, media):
    
    filter1 = d['publication'] == media
    news_data = d.loc[filter1]
    return news_data
    
    

In [181]:
#text = news_articles[1]
#text = "WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been demanding an end to the law for years. In another twist, Donald J. Trump’s administration, worried about preserving executive branch prerogatives, could choose to fight its Republican allies in the House on some central questions in the dispute. Eager to avoid an ugly political pileup, Republicans on Capitol Hill and the Trump transition team are gaming out how to handle the lawsuit, which, after the election, has been put in limbo until at least late February by the United States Court of Appeals for the District of Columbia Circuit. They are not yet ready to divulge their strategy. “Given that this pending litigation involves the Obama administration and Congress, it would be inappropriate to comment,” said Phillip J. Blando, a spokesman for the Trump transition effort. “Upon taking office, the Trump administration will evaluate this case and all related aspects of the Affordable Care Act. ”"
#print(text)
#sent_resolved = get_resolved_coref_text(text)
#print("unresolved text \n ", text)
#print("coref Resolved Text \n ", sent_resolved)
#sys.exit()

def processed_text(d, count):    

    file_name = 'coref_resolved_news_articles.csv'
    #count = 0
    for index, row in d.iterrows():
        df = data.iloc[[index]]
        text = row['content']
        #text = df.loc[:,'content']
        #text = str(text)
        #print("len ", len(text))
        #print(text)
        #print(df)
        #text = str(row['content'])
        #sys.exit()
        
        #print(index)
        #sys.exit()
        #print("Original text ", text)

        #print("NEEEER ", get_ner_tags(text))
        #sys.exit()
        resolved_text = get_resolved_coref_text(text)
        #spacy_ner = get_spacy_ner(text)
        df.iloc[:,-1] = resolved_text  
        #print("Resolved_text ", resolved_text)
        #pprint(ner)
        #print("\n")
        #pprint(sp_ner)
        df.to_csv(file_name, mode='a', header=False)
        #d = pd.read_csv(file_name)
        #print(d.shape)
        print("count = ", count)
        count+=1
        #sys.exit()


count = 41045
d = data.iloc[count:,]
#print(d.shape, data.shape)
processed_text(d, count)
    

count =  41045
count =  41046
count =  41047
count =  41048
count =  41049
count =  41050
count =  41051
count =  41052
count =  41053
count =  41054
count =  41055
count =  41056
count =  41057
count =  41058
count =  41059
count =  41060
count =  41061
count =  41062
count =  41063
count =  41064
count =  41065
count =  41066
count =  41067
count =  41068
count =  41069
count =  41070
count =  41071
count =  41072
count =  41073
count =  41074
count =  41075
count =  41076
count =  41077
count =  41078
count =  41079
count =  41080
count =  41081
count =  41082
count =  41083
count =  41084
count =  41085
count =  41086
count =  41087
count =  41088
count =  41089
count =  41090
count =  41091
count =  41092
count =  41093
count =  41094
count =  41095
count =  41096
count =  41097
count =  41098
count =  41099
count =  41100
count =  41101
count =  41102
count =  41103
count =  41104
count =  41105
count =  41106
count =  41107
count =  41108
count =  41109
count =  41110
count =  4

count =  41592
count =  41593
count =  41594
count =  41595
count =  41596
count =  41597
count =  41598
count =  41599
count =  41600
count =  41601
count =  41602
count =  41603
count =  41604
count =  41605
count =  41606
count =  41607
count =  41608
count =  41609
count =  41610
count =  41611
count =  41612
count =  41613
count =  41614
count =  41615
count =  41616
count =  41617
count =  41618
count =  41619
count =  41620
count =  41621
count =  41622
count =  41623
count =  41624
count =  41625
count =  41626
count =  41627
count =  41628
count =  41629
count =  41630
count =  41631
count =  41632
count =  41633
count =  41634
count =  41635
count =  41636
count =  41637
count =  41638
count =  41639
count =  41640
count =  41641
count =  41642
count =  41643
count =  41644
count =  41645
count =  41646
count =  41647
count =  41648
count =  41649
count =  41650
count =  41651
count =  41652
count =  41653
count =  41654
count =  41655
count =  41656
count =  41657
count =  4

count =  42139
count =  42140
count =  42141
count =  42142
count =  42143
count =  42144
count =  42145
count =  42146
count =  42147
count =  42148
count =  42149
count =  42150
count =  42151
count =  42152
count =  42153
count =  42154
count =  42155
count =  42156
count =  42157
count =  42158
count =  42159
count =  42160
count =  42161
count =  42162
count =  42163
count =  42164
count =  42165
count =  42166
count =  42167
count =  42168
count =  42169
count =  42170
count =  42171
count =  42172
count =  42173
count =  42174
count =  42175
count =  42176
count =  42177
count =  42178
count =  42179
count =  42180
count =  42181
count =  42182
count =  42183
count =  42184
count =  42185
count =  42186
count =  42187
count =  42188
count =  42189
count =  42190
count =  42191
count =  42192
count =  42193
count =  42194
count =  42195
count =  42196
count =  42197
count =  42198
count =  42199
count =  42200
count =  42201
count =  42202
count =  42203
count =  42204
count =  4

count =  42686
count =  42687
count =  42688
count =  42689
count =  42690
count =  42691
count =  42692
count =  42693
count =  42694
count =  42695
count =  42696
count =  42697
count =  42698
count =  42699
count =  42700
count =  42701
count =  42702
count =  42703
count =  42704
count =  42705
count =  42706
count =  42707
count =  42708
count =  42709
count =  42710
count =  42711
count =  42712
count =  42713
count =  42714
count =  42715
count =  42716
count =  42717
count =  42718
count =  42719
count =  42720
count =  42721
count =  42722
count =  42723
count =  42724
count =  42725
count =  42726
count =  42727
count =  42728
count =  42729
count =  42730
count =  42731
count =  42732
count =  42733
count =  42734
count =  42735
count =  42736
count =  42737
count =  42738
count =  42739
count =  42740
count =  42741
count =  42742
count =  42743
count =  42744
count =  42745
count =  42746
count =  42747
count =  42748
count =  42749
count =  42750
count =  42751
count =  4

count =  43233
count =  43234
count =  43235
count =  43236
count =  43237
count =  43238
count =  43239
count =  43240
count =  43241
count =  43242
count =  43243
count =  43244
count =  43245
count =  43246
count =  43247
count =  43248
count =  43249
count =  43250
count =  43251
count =  43252
count =  43253
count =  43254
count =  43255
count =  43256
count =  43257
count =  43258
count =  43259
count =  43260
count =  43261
count =  43262
count =  43263
count =  43264
count =  43265
count =  43266
count =  43267
count =  43268
count =  43269
count =  43270
count =  43271
count =  43272
count =  43273
count =  43274
count =  43275
count =  43276
count =  43277
count =  43278
count =  43279
count =  43280
count =  43281
count =  43282
count =  43283
count =  43284
count =  43285
count =  43286
count =  43287
count =  43288
count =  43289
count =  43290
count =  43291
count =  43292
count =  43293
count =  43294
count =  43295
count =  43296
count =  43297
count =  43298
count =  4

...

count =  47609
count =  47610
count =  47611
count =  47612
count =  47613
count =  47614
count =  47615
count =  47616
count =  47617
count =  47618
count =  47619
count =  47620
count =  47621
count =  47622
count =  47623
count =  47624
count =  47625
count =  47626
count =  47627
count =  47628
count =  47629
count =  47630
count =  47631
count =  47632
count =  47633
count =  47634
count =  47635
count =  47636
count =  47637
count =  47638
count =  47639
count =  47640
count =  47641
count =  47642
count =  47643
count =  47644
count =  47645
count =  47646
count =  47647
count =  47648
count =  47649
count =  47650
count =  47651
count =  47652
count =  47653
count =  47654
count =  47655
count =  47656
count =  47657
count =  47658
count =  47659
count =  47660
count =  47661
count =  47662
count =  47663
count =  47664
count =  47665
count =  47666
count =  47667
count =  47668
count =  47669
count =  47670
count =  47671
count =  47672
count =  47673
count =  47674
count =  4

count =  48156
count =  48157
count =  48158
count =  48159
count =  48160
count =  48161
count =  48162
count =  48163
count =  48164
count =  48165
count =  48166
count =  48167
count =  48168
count =  48169
count =  48170
count =  48171
count =  48172
count =  48173
count =  48174
count =  48175
count =  48176
count =  48177
count =  48178
count =  48179
count =  48180
count =  48181
count =  48182
count =  48183
count =  48184
count =  48185
count =  48186
count =  48187
count =  48188
count =  48189
count =  48190
count =  48191
count =  48192
count =  48193
count =  48194
count =  48195
count =  48196
count =  48197
count =  48198
count =  48199
count =  48200
count =  48201
count =  48202
count =  48203
count =  48204
count =  48205
count =  48206
count =  48207
count =  48208
count =  48209
count =  48210
count =  48211
count =  48212
count =  48213
count =  48214
count =  48215
count =  48216
count =  48217
count =  48218
count =  48219
count =  48220
count =  48221
count =  4

count =  48703
count =  48704
count =  48705
count =  48706
count =  48707
count =  48708
count =  48709
count =  48710
count =  48711
count =  48712
count =  48713
count =  48714
count =  48715
count =  48716
count =  48717
count =  48718
count =  48719
count =  48720
count =  48721
count =  48722
count =  48723
count =  48724
count =  48725
count =  48726
count =  48727
count =  48728
count =  48729
count =  48730
count =  48731
count =  48732
count =  48733
count =  48734
count =  48735
count =  48736
count =  48737
count =  48738
count =  48739
count =  48740
count =  48741
count =  48742
count =  48743
count =  48744
count =  48745
count =  48746
count =  48747
count =  48748
count =  48749
count =  48750
count =  48751
count =  48752
count =  48753
count =  48754
count =  48755
count =  48756
count =  48757
count =  48758
count =  48759
count =  48760
count =  48761
count =  48762
count =  48763
count =  48764
count =  48765
count =  48766
count =  48767
count =  48768
count =  4

count =  49250
count =  49251
count =  49252
count =  49253
count =  49254
count =  49255
count =  49256
count =  49257
count =  49258
count =  49259
count =  49260
count =  49261
count =  49262
count =  49263
count =  49264
count =  49265
count =  49266
count =  49267
count =  49268
count =  49269
count =  49270
count =  49271
count =  49272
count =  49273
count =  49274
count =  49275
count =  49276
count =  49277
count =  49278
count =  49279
count =  49280
count =  49281
count =  49282
count =  49283
count =  49284
count =  49285
count =  49286
count =  49287
count =  49288
count =  49289
count =  49290
count =  49291
count =  49292
count =  49293
count =  49294
count =  49295
count =  49296
count =  49297
count =  49298
count =  49299
count =  49300
count =  49301
count =  49302
count =  49303
count =  49304
count =  49305
count =  49306
count =  49307
count =  49308
count =  49309
count =  49310
count =  49311
count =  49312
count =  49313
count =  49314
count =  49315
count =  4

count =  49797
count =  49798
count =  49799
count =  49800
count =  49801
count =  49802
count =  49803
count =  49804
count =  49805
count =  49806
count =  49807
count =  49808
count =  49809
count =  49810
count =  49811
count =  49812
count =  49813
count =  49814
count =  49815
count =  49816
count =  49817
count =  49818
count =  49819
count =  49820
count =  49821
count =  49822
count =  49823
count =  49824
count =  49825
count =  49826
count =  49827
count =  49828
count =  49829
count =  49830
count =  49831
count =  49832
count =  49833
count =  49834
count =  49835
count =  49836
count =  49837
count =  49838
count =  49839
count =  49840
count =  49841
count =  49842
count =  49843
count =  49844
count =  49845
count =  49846
count =  49847
count =  49848
count =  49849
count =  49850
count =  49851
count =  49852
count =  49853
count =  49854
count =  49855
count =  49856
count =  49857
count =  49858
count =  49859
count =  49860
count =  49861
count =  49862
count =  4

In [ ]:
from nltk import sent_tokenize
s1 = sent_tokenize(text)
s2 = sent_tokenize(sent_resolved)
for s in s1:
    print(s, "\n")
for s in s2:
    print(s, "\n")